In [13]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from pprint import pprint
import googlemaps
import json
import os
import requests

In [14]:
API_KEY= os.getenv('GOOGLE_MAP_API_KEY')
gmap = googlemaps.Client(API_KEY)

Get Yelp Hotel info

In [15]:
hotels = []

# Open the file and read it line by line
with open('Datasets/Hotels.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels.append(json_obj)

assert len(hotels) == 107

Need to preprocess the address, take the address before the "," or google won't return any candidates 

use Yelp business name, address, and city to get the google id

In [ ]:
#get business id using find place from text
hotels_google = []

for hotel in hotels:
    name = hotel['name']
    address = hotel['address']
    city = hotel['city']

    query = f'{name}, {address}, {city}'

    url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=formatted_address%2Cname%2Cplace_id&input={query}&inputtype=textquery&key={API_KEY}'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        hotels_google.append(data)
    else:
        print(f"Error for {name}:", response.status_code)

In [ ]:
hotels_google[3]

In [12]:
with open('Hotels_Google_Candidates.jsonl', 'w') as json_file:    
    for item in hotels_google:
        json.dump(item, json_file)
        json_file.write('\n')

formatting the candidates jsonl, and only keep the first candidates, (if multiple candidates returned, that mean the original business already closed, the first candidates is the most up to date business at the location, even though the old business won't get the place_id and information from google map anymore, it's okay, since they are closed and won't matter the current travel plan)

In [18]:
hotels_google = []


# Open the file and read it line by line
with open('Datasets/Hotels_Google_Candidates.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        hotels_google.append(json_obj)

In [20]:
hotels_google_clean = []
for hotel_google in hotels_google:
    candidates = hotel_google['candidates']
    if(len(candidates)):
        hotels_google_clean.append(candidates[0]) #only keep the first candidates
    else:
        hotels_google_clean.append({})

#write to jsonl file
with open('Datasets/Hotels_Google_Candidates_clean.jsonl', 'w') as file:
    for hotel_google in hotels_google_clean:
        json.dump(hotel_google, file)
        file.write('\n')

get actual google map information using place id

In [22]:
#read jsonl file 'Hotels_Google_Candidates_clean.jsonl'
hotels_google_clean = []
with open('Datasets/Hotels_Google_Candidates_clean.jsonl', 'r') as file:
    for line in file:
        hotels_google_clean.append(json.loads(line))
hotels_google_clean #if no candidates, return a {} as a place holder

[{'formatted_address': '4301 Island Ave, Philadelphia, PA 19153, United States',
  'name': 'Aloft Philadelphia Airport',
  'place_id': 'ChIJKQQQrGTExokRpOHLLAkvFNw'},
 {'formatted_address': '1450 Sansom St, Philadelphia, PA 19102, United States',
  'name': 'Inn At The Union League',
  'place_id': 'ChIJOXIvky_GxokRzq3LEQWsxjE'},
 {'formatted_address': '4444 City Ave, Philadelphia, PA 19131, United States',
  'name': 'Philly Inn & Suites',
  'place_id': 'ChIJu-FagWHHxokRcJNCZWaIpDM'},
 {'formatted_address': '433 Chestnut St, Philadelphia, PA 19106, United States',
  'name': 'Kimpton Hotel Monaco Philadelphia',
  'place_id': 'ChIJN5P3moTIxokREvdtKFMo3Sk'},
 {'formatted_address': '237 S Broad St, Philadelphia, PA 19107, United States',
  'name': 'DoubleTree by Hilton Hotel Philadelphia Center City',
  'place_id': 'ChIJQ433EiXGxokRnWqMZZoUCDo'},
 {'formatted_address': '7600 Roosevelt Blvd # 1, Philadelphia, PA 19152, United States',
  'name': 'Roosevelt Inn',
  'place_id': 'ChIJxX9IT42zxokR

In [30]:
def GetAttributesUsingPlaceId(place_id):
    url = f'https://places.googleapis.com/v1/places/{place_id}?fields=*&key={API_KEY}'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return response.status_code

In [31]:
hotel_google_attributes = []

for hotel_google in hotels_google_clean:
    if(len(hotel_google)):
        place_id = hotel_google['place_id']
        attributes = GetAttributesUsingPlaceId(place_id)
        hotel_google_attributes.append(attributes)
    else:
        hotel_google_attributes.append({})

In [34]:
#write into Hotels_Google_Attributes.jsonl
with open('Datasets/Hotels_Google_Attributes.jsonl', 'w') as json_file:    
    for item in hotel_google_attributes:
        json.dump(item, json_file)
        json_file.write('\n')